In [1]:
import pandas as pd
from google import genai
import re
from dotenv import load_dotenv
import os

In [2]:
df = pd.read_csv('Game Thumbnail.csv')
api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
# df = df.head(5)
df.shape

(50, 2)

In [4]:
def genre_classifier(titles, client=None, batch_size=5):
    """
    Classifies multiple video game titles into genres and returns results ready for DataFrame.
    
    Args:
        titles (list or pandas.Series): List or Series of video game titles to classify
        client (genai.Client, optional): The Gemini API client. If None, uses the global client.
        batch_size (int): Number of titles to process in each batch
        
    Returns:
        list: List of genre classifications corresponding to input titles
    """
    # Use provided client or fall back to global client
    if client is None:
        from google import genai
        client_to_use = genai.Client(api_key=api_key)
    else:
        client_to_use = client
    
    # Convert to list if it's a pandas Series
    if hasattr(titles, 'tolist'):
        titles = titles.tolist()
    
    results = []
    
    # Process in batches to avoid API rate limits
    for i in range(0, len(titles), batch_size):
        batch = titles[i:i+batch_size]
        
        # Create a prompt that handles multiple titles at once
        prompt = "Classify each of the following video games into a single-word genre category. For each game, respond with ONLY the genre name, one per line, in the same order as the games listed:\n\n"
        for title in batch:
            prompt += f"- {title}\n"
        
        response = client_to_use.models.generate_content(
            model='gemini-2.0-flash-lite',
            contents=prompt
        )
        
        # Parse the response - expecting one genre per line
        genres = response.text.strip().split('\n')
        # Clean up any bullet points or numbering
        genres = [g.strip().lstrip('-').lstrip('*').lstrip('1234567890.').strip() for g in genres]
        
        # Only take as many results as we had titles
        results.extend(genres[:len(batch)])
    
    return results

df['genre'] = genre_classifier(df['game_title'])

In [5]:
df.head()

,game_title,image_url,genre
0,Street Fighter 6,https://images.igdb.com/igdb/image/upload/t_co...,Fighting
1,Hunt: Showdown 1896,https://images.igdb.com/igdb/image/upload/t_co...,Survival
2,Wuthering Waves,https://images.igdb.com/igdb/image/upload/t_co...,Action
3,Arma Reforger,https://images.igdb.com/igdb/image/upload/t_co...,Tactical
4,Lethal Company,https://images.igdb.com/igdb/image/upload/t_co...,Survival


In [ ]:
def gen_short_desc(titles, client=None, batch_size=5):
    """
    Generates a short description for a video game using the given title.
    Args:
        titles (list or pandas.Series): List or Series of video game titles to classify
        client (genai.Client, optional): The Gemini API client. If None, uses the global client.
        batch_size (int): Number of titles to process in each batch
        
    Returns:
        list: List of short descriptions corresponding to input titles
    """
    # Use provided client or fall back to global client
    if client is None:
        from google import genai
        client_to_use = genai.Client(api_key=api_key)
    else:
        client_to_use = client
    
    # Convert to list if it's a pandas Series
    if hasattr(titles, 'tolist'):
        titles = titles.tolist()
    
    results = []
    
    # Process in batches to avoid API rate limits
    for i in range(0, len(titles), batch_size):
        batch = titles[i:i+batch_size]
        
        # Create a prompt that handles multiple titles at once
        prompt = "Generate short descriptions for each of the following video games. For each game, respond with ONLY the short description under 30 words, each in one line, in the same order as the games listed:\n\n"
        for title in batch:
            prompt += f"- {title}\n"
        
        response = client_to_use.models.generate_content(
            model='gemini-2.0-flash-lite',
            contents=prompt
        )
        
        # Parse the response - expecting one genre per line
        descriptions = response.text.strip().split('\n')
        # Clean up any bullet points or numbering
        descriptions = [d.strip().lstrip('-').lstrip('*') for d in descriptions]
        descriptions = [re.sub(r'^(\d+)[\.\)]?\s*', '', d).strip() for d in descriptions]
        
        # Only take as many results as we had titles
        results.extend(descriptions[:len(batch)])

    print(results)
    return results

df['short_description'] = gen_short_desc(df['game_title'])

['Street Fighter 6:', 'Next-gen fighting with diverse characters, engaging modes, and a modern, accessible experience.', '', 'Hunt: Showdown 1896:', 'Brutal PvPvE bounty hunting in a gothic, atmospheric Louisiana swamp. Fight monsters and players for glory.', 'Arena Breakout: Infinite: A tactical extraction FPS where you raid warzones, loot valuable gear, and fight to survive.', 'Zenless Zone Zero: A stylish action RPG set in a post-apocalyptic city, battling monsters with unique characters.', 'ARK: Survival Ascended: Survive a prehistoric, hostile island filled with dinosaurs, craft, build, and tame creatures.', "Sid Meier's Civilization VII: Build an empire to stand the test of time, make pivotal decisions to expand your empire.", 'SMITE 2: A revamped MOBA featuring gods battling for dominance, with updated visuals and gameplay.', 'A free-to-play MMORPG with dynamic combat, world transformation, and seamless transitions between areas.', 'Survive and build in a vast, voxel-based open 

In [7]:
df

,game_title,image_url,genre,short_description
0,Street Fighter 6,https://images.igdb.com/igdb/image/upload/t_co...,Fighting,Street Fighter 6:
1,Hunt: Showdown 1896,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"Next-gen fighting with diverse characters, eng..."
2,Wuthering Waves,https://images.igdb.com/igdb/image/upload/t_co...,Action,
3,Arma Reforger,https://images.igdb.com/igdb/image/upload/t_co...,Tactical,Hunt: Showdown 1896:
4,Lethal Company,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"Brutal PvPvE bounty hunting in a gothic, atmos..."
5,Arena Breakout: Infinite,https://images.igdb.com/igdb/image/upload/t_co...,Shooter,Arena Breakout: Infinite: A tactical extractio...
6,Zenless Zone Zero,https://images.igdb.com/igdb/image/upload/t_co...,RPG,Zenless Zone Zero: A stylish action RPG set in...
7,ARK: Survival Ascended,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"ARK: Survival Ascended: Survive a prehistoric,..."
8,Sid Meier's Civilization VII,https://images.igdb.com/igdb/image/upload/t_co...,Strategy,Sid Meier's Civilization VII: Build an empire ...
9,SMITE 2,https://images.igdb.com/igdb/image/upload/t_co...,MOBA,SMITE 2: A revamped MOBA featuring gods battli...


In [8]:
df['short_description']

0                                     Street Fighter 6:
1     Next-gen fighting with diverse characters, eng...
2                                                      
3                                  Hunt: Showdown 1896:
4     Brutal PvPvE bounty hunting in a gothic, atmos...
5     Arena Breakout: Infinite: A tactical extractio...
6     Zenless Zone Zero: A stylish action RPG set in...
7     ARK: Survival Ascended: Survive a prehistoric,...
8     Sid Meier's Civilization VII: Build an empire ...
9     SMITE 2: A revamped MOBA featuring gods battli...
10    A free-to-play MMORPG with dynamic combat, wor...
11    Survive and build in a vast, voxel-based open ...
12    Survive the horrors within Playtime Co., solvi...
13    A co-op adventure where two players must work ...
14    Experience epic, one-versus-thousand battles a...
15    Skate, shoot, and brawl your way to the Stanle...
16    A deck-building roguelike where you bluff your...
17    Explore an adorable 3D world, make friends

In [ ]:
def player_mode_classifier(titles, client=None, batch_size=5):
    """
    Classifies the player mode of multiple video game titles.
    
    Args:
        titles (list or pandas.Series): List or Series of video game titles to classify
        client (genai.Client, optional): The Gemini API client. If None, uses the global client.
        batch_size (int): Number of titles to process in each batch
        
    Returns:
        list: List of player mode classifications corresponding to input titles
    """
    # Use provided client or fall back to global client
    if client is None:
        from google import genai
        client_to_use = genai.Client(api_key="api_key")
    else:
        client_to_use = client
    
    # Convert to list if it's a pandas Series
    if hasattr(titles, 'tolist'):
        titles = titles.tolist()
    
    results = []
    
    # Process in batches to avoid API rate limits
    for i in range(0, len(titles), batch_size):
        batch = titles[i:i+batch_size]
        
        # Create a prompt that handles multiple titles at once
        prompt = "Classify each of the following video games based on their player mode. For each game, respond with ONLY one of these three options: 'Singleplayer', 'Multiplayer', or 'Both'. A game should be classified as 'Both' if it includes any cooperative or online multiplayer features, even if primarily designed for single-player. Respond with one classification per line, in the same order as the games listed:\n\n"
        for title in batch:
            prompt += f"- {title}\n"
        
        response = client_to_use.models.generate_content(
            model='gemini-2.0-flash-lite',
            contents=prompt
        )
        
        # Parse the response - expecting one genre per line
        player_mode = response.text.strip().split('\n')
        # Clean up any bullet points or numbering
        player_mode = [p.strip().lstrip('-').lstrip('*').lstrip('1234567890.').strip() for p in player_mode]
        
        # Only take as many results as we had titles
        results.extend(player_mode[:len(batch)])
    
    return results

df['player_mode'] = player_mode_classifier(df['game_title'])

In [10]:
df

,game_title,image_url,genre,short_description,player_mode
0,Street Fighter 6,https://images.igdb.com/igdb/image/upload/t_co...,Fighting,Street Fighter 6:,Multiplayer
1,Hunt: Showdown 1896,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"Next-gen fighting with diverse characters, eng...",Multiplayer
2,Wuthering Waves,https://images.igdb.com/igdb/image/upload/t_co...,Action,,Both
3,Arma Reforger,https://images.igdb.com/igdb/image/upload/t_co...,Tactical,Hunt: Showdown 1896:,Multiplayer
4,Lethal Company,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"Brutal PvPvE bounty hunting in a gothic, atmos...",Multiplayer
5,Arena Breakout: Infinite,https://images.igdb.com/igdb/image/upload/t_co...,Shooter,Arena Breakout: Infinite: A tactical extractio...,Multiplayer
6,Zenless Zone Zero,https://images.igdb.com/igdb/image/upload/t_co...,RPG,Zenless Zone Zero: A stylish action RPG set in...,Both
7,ARK: Survival Ascended,https://images.igdb.com/igdb/image/upload/t_co...,Survival,"ARK: Survival Ascended: Survive a prehistoric,...",Both
8,Sid Meier's Civilization VII,https://images.igdb.com/igdb/image/upload/t_co...,Strategy,Sid Meier's Civilization VII: Build an empire ...,Singleplayer
9,SMITE 2,https://images.igdb.com/igdb/image/upload/t_co...,MOBA,SMITE 2: A revamped MOBA featuring gods battli...,Multiplayer
